<a href="https://colab.research.google.com/github/Vasanth19/Biz121/blob/master/mcPaul_Instagram_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Input Area

file_name = '/content/bd-similar-creators - Carousels.csv' # @param {type:'string'}

drive_folder = '/content/drive/MyDrive/Backups' # @param {type:'string'}

In [ ]:
# @title Code Area

import warnings
import os
import shutil
import json
import zipfile
from tqdm import tqdm
import requests
import pandas as pd

warnings.filterwarnings('ignore')

df = pd.read_csv(file_name)
posts = df['url'].tolist()
print(f'Found {len(posts)} posts', end='\n\n')


def get_data(url):
    endpoint = 'https://fastdl.app/api/convert'

    payload = json.dumps({
      'url': url
    })

    headers = {
      'accept': 'application/json, text/plain, */*',
      'accept-language': 'en-US,en;q=0.9,tr;q=0.8',
      'cache-control': 'no-cache',
      'content-type': 'application/json',
      'origin': 'https://fastdl.app',
      'pragma': 'no-cache',
      'referer': 'https://fastdl.app/en',
      'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Mobile Safari/537.36'
    }

    response = requests.request('POST', endpoint, headers=headers, data=payload)
    assert response.status_code == 200
    return response.json()


folder_names = []


for post in tqdm(posts):
    try:
        data = get_data(post)
    except:
        continue

    try:
        title = data[0]['meta']['title']
    except:
        title = data['meta']['title']

    folder_name = title[:20]
    os.makedirs(folder_name, exist_ok=True)

    if type(data) != list:
        try:
            url = data['url'][0]['url']
            extention = data['url'][0]['ext']

            r = requests.get(url)

            with open(os.path.join(folder_name, f'1.{extention}'), 'wb') as f:
                f.write(r.content)

            folder_names.append(folder_name)
        except:
            continue
    else:
        try:
            for idx, item in enumerate(data):
                url = item['url'][0]['url']
                extention = item['url'][0]['ext']

                r = requests.get(url)

                with open(os.path.join(folder_name, f'{idx + 1}.{extention}'), 'wb') as f:
                    f.write(r.content)

            folder_names.append(folder_name)
        except:
            continue


with zipfile.ZipFile('Instagram Downloader Output.zip', 'w') as zip_file:
    for folder in folder_names:
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.join(folder, '..'))
                zip_file.write(file_path, arcname)


for folder in folder_names:
    try:
        shutil.rmtree(folder)
    except:
        pass


shutil.move('Instagram Downloader Output.zip', drive_folder)
print(f'\nDownloaded {len(posts)} posts and zipped them into Instagram Downloader Output.zip')